In [ ]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/FEVER

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/FEVER'
/content


In [ ]:
!pip install tqdm
!pip install nltk
!pip install torch
!pip install scipy
!pip install transformers
!pip install sklearn

  Using cached sklearn-0.0.post10.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post10-py3-none-any.whl size=2959 sha256=152ffc97a1c773bd4efa2c4a0493c1c6bc0e15514da93fd7044ab11c8335eaea
  Stored in directory: /root/.cache/pip/wheels/5b/f6/92/0173054cc528db7ffe7b0c7652a96c3102aab156a6da960387
Successfully built sklearn


In [ ]:
# a copy from run_classfier to test tokenizer


# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""BERT finetuning runner."""

from __future__ import absolute_import, division, print_function

import pickle
import argparse
import csv
import logging
import os
import random
import sys
import json

import pandas as pd
import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler, TensorDataset, ConcatDataset, Subset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import trange
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss, BCELoss
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef,accuracy_score, f1_score
from sklearn.model_selection import KFold

import transformers
from transformers import RobertaForSequenceClassification, RobertaConfig, BertTokenizer
from transformers import PYTORCH_PRETRAINED_BERT_CACHE, WEIGHTS_NAME, CONFIG_NAME
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification
from transformers import RobertaModel

logger = logging.getLogger(__name__)


# Metric eval

In [ ]:

# METRICS EVAL
def acc_and_f1(preds, labels):
    acc = accuracy_score(y_true=labels, y_pred=preds)
    f1 = f1_score(y_true=labels, y_pred=preds, average="macro")
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }

def pearson_and_spearman(preds, labels):
    pearson_corr = pearsonr(preds, labels)[0]
    spearman_corr = spearmanr(preds, labels)[0]
    return {
        "pearson": pearson_corr,
        "spearmanr": spearman_corr,
        "corr": (pearson_corr + spearman_corr) / 2,
    }


# Arguments

In [ ]:
class Arguments:
    def __init__(self):
        self.data_dir = "/content/drive/MyDrive/pRoBERTa/data/data"
        self.bert_model = "vinai/phobert-base-v2"
        self.task_name = "fever"
        self.output_dir = "/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/self_explain_model"
        self.evidence_path = "/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/self_explain_model/pytorch_model.bin"
        self.kfold = 10
        self.scale_pos_neg = 5
        self.cache_dir = ""
        self.max_seq_length = 128
        self.do_train = True
        self.do_eval = True
        self.do_test = False
        self.do_lower_case = False
        self.train_batch_size = 2
        self.negative_batch_size = 4
        self.losstype = 'cross_entropy_mining'
        self.eval_batch_size = 6
        self.learning_rate = 2e-5
        self.num_train_epochs = 5.0
        self.warmup_proportion = 0.1
        self.no_cuda = False
        self.local_rank = -1
        self.seed = 42
        self.gradient_accumulation_steps = 1
        self.fp16 = False
        self.loss_scale = 0.0
args=Arguments()


# Model


## Colate function


In [ ]:
from typing import List

import numpy as np
import torch

def collate_to_max_length(batch: List[List[torch.Tensor]], max_len: int = 128, fill_values: List[float] = None) -> \
    List[torch.Tensor]:
    """
    pad to maximum length of this batch
    Args:
        batch: a batch of samples, each contains a list of field data(Tensor), which shape is [seq_length]
        max_len: specify max length
        fill_values: specify filled values of each field
    Returns:
        output: list of field batched data, which shape is [batch, max_length]
    """
    # [batch, num_fields]
    lengths = np.array([[len(field_data) for field_data in sample] for sample in batch])
    batch_size, num_fields = lengths.shape
    fill_values = fill_values or [0.0] * num_fields
    # [num_fields]
    max_lengths = max_len
    # if max_len:
    #     assert max_lengths.max() <= max_len
    #     max_lengths = np.ones_like(max_lengths) * max_len

    output = [torch.full([batch_size, max_lengths],
                         fill_value=fill_values[field_idx],
                         dtype=batch[0][field_idx].dtype)
              for field_idx in range(num_fields)]
    for sample_idx in range(batch_size):
        for field_idx in range(num_fields):
            # seq_length
            data = batch[sample_idx][field_idx]
            output[field_idx][sample_idx][: data.shape[0]] = data
    # generate span_index and span_mask
    max_sentence_length = max_lengths
    start_indexs = []
    end_indexs = []
    for i in range(1, max_sentence_length - 1):
        for j in range(i, max_sentence_length - 1):
            # # span大小为10
            # if j - i > 10:
            #     continue
            start_indexs.append(i)
            end_indexs.append(j)
    # generate span mask
    span_masks = []
    for input_ids, label, length in batch:
        span_mask = []
        middle_index = input_ids.tolist().index(2)
        for start_index, end_index in zip(start_indexs, end_indexs):
            if 1 <= start_index <= length.item() - 2 and 1 <= end_index <= length.item() - 2 and (
                start_index > middle_index or end_index < middle_index):
                span_mask.append(0)
            else:
                span_mask.append(1e6)
        span_masks.append(span_mask)
    # add to output
    output.append(torch.LongTensor(start_indexs))
    output.append(torch.LongTensor(end_indexs))
    output.append(torch.LongTensor(span_masks))
    return output  # (input_ids, labels, length, start_indexs, end_indexs, span_masks)



## DataLoader


In [ ]:
import json
import os
from functools import partial

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer

class DSC_Dataset(Dataset):
    def __init__(self, directory, prefix, bert_path, max_length: int = 128):
        super().__init__()
        self.max_length = max_length
        # label_map = {"NON": 0, 'SR': 1}
        with open(os.path.join(directory, prefix + '.json')) as file:
            data = json.load(file)
        self.result = data
        self.tokenizer = AutoTokenizer.from_pretrained(args.bert_model)

    def __len__(self):
        return len(self.result)

    def __getitem__(self, idx):
        # sentence_1, sentence_2, label = self.result[idx]
        # check for required keys
        if not all(key in self.result[idx] for key in ["claim", "sentence", "evidence_label"]):
            raise KeyError("Missing key in result dictionary.")

        sentence_1 = self.result[idx]["claim"]
        sentence_2 = self.result[idx]["sentence"]
        label = self.result[idx]["evidence_label"]
        # remove .
        if sentence_1.endswith("."):
            sentence_1 = sentence_1[:-1]
        if sentence_2.endswith("."):
            sentence_2 = sentence_2[:-1]
        sentence_1_input_ids = self.tokenizer.encode(
            sentence_1, add_special_tokens=False)
        sentence_2_input_ids = self.tokenizer.encode(
            sentence_2, add_special_tokens=False)
        input_ids = sentence_1_input_ids + [2] + sentence_2_input_ids
        if len(input_ids) > self.max_length - 2:
            input_ids = input_ids[:self.max_length - 2]
        # convert list to tensor
        length = torch.LongTensor([len(input_ids) + 2])
        input_ids = torch.LongTensor([0] + input_ids + [2])
        label = torch.LongTensor([label])
        return input_ids, label, length


# def unit_test():
#     root_path = "D:\\DUCBUI\\DSC\\self_explanining\\data\\"
#     bert_path = "roberta-base"
#     prefix = "train_evidence"
#     dataset = DSC_Dataset(directory=root_path,
#                           prefix=prefix, bert_path=bert_path)

#     print(dataset)



# if __name__ == '__main__':
#     unit_test()


In [ ]:
def get_feature(dataset, sampler, batch_size ) -> DataLoader:
    dataloader = DataLoader(
        dataset=dataset,
        sampler=sampler,
        batch_size=batch_size,
        collate_fn=partial(collate_to_max_length, fill_values=[1, 0, 0]),
        drop_last=False
    )
    return dataloader

    # iterator = iter(dataloader)
    # batch = next(iterator)
    # input_ids, label, length, start_index, end_index, span_mask = batch
    # return input_ids, label, length, start_index, end_index, span_mask

## Explainable model

In [ ]:
class ExplainableModel(nn.Module):
    def __init__(self, bert_dir):
        super().__init__()
        self.bert_config = RobertaConfig.from_pretrained(
            bert_dir, output_hidden_states=True)
        self.intermediate = RobertaModel.from_pretrained(bert_dir)
        self.span_info_collect = SICModel(self.bert_config.hidden_size)
        self.interpretation = InterpretationModel(self.bert_config.hidden_size)
        self.output = nn.Linear(
            self.bert_config.hidden_size, self.bert_config.num_labels)

    def forward(self, input_ids, start_indexs, end_indexs, span_masks):
        # generate mask
        attention_mask = (input_ids != 1).long()
        # intermediate layer
        a = self.intermediate(
            input_ids, attention_mask=attention_mask)
        hidden_states = self.intermediate(
            input_ids, attention_mask=attention_mask).last_hidden_state
        # span info collecting layer(SIC)
        h_ij = self.span_info_collect(hidden_states, start_indexs, end_indexs)
        # interpretation layer
        H, a_ij = self.interpretation(h_ij, span_masks)
        # output layer
        out = self.output(H)
        return out, a_ij


class SICModel(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.W_1 = nn.Linear(hidden_size, hidden_size)
        self.W_2 = nn.Linear(hidden_size, hidden_size)
        self.W_3 = nn.Linear(hidden_size, hidden_size)
        self.W_4 = nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden_states, start_indexs, end_indexs):
        W1_h = self.W_1(hidden_states)  # (bs, length, hidden_size)
        W2_h = self.W_2(hidden_states)
        W3_h = self.W_3(hidden_states)
        W4_h = self.W_4(hidden_states)

        # (bs, span_num, hidden_size)
        W1_hi_emb = torch.index_select(W1_h, 1, start_indexs)
        W2_hj_emb = torch.index_select(W2_h, 1, end_indexs)
        W3_hi_start_emb = torch.index_select(W3_h, 1, start_indexs)
        W3_hi_end_emb = torch.index_select(W3_h, 1, end_indexs)
        W4_hj_start_emb = torch.index_select(W4_h, 1, start_indexs)
        W4_hj_end_emb = torch.index_select(W4_h, 1, end_indexs)

        # [w1*hi, w2*hj, w3(hi-hj), w4(hi⊗hj)]
        span = W1_hi_emb + W2_hj_emb + \
            (W3_hi_start_emb - W3_hi_end_emb) + \
            torch.mul(W4_hj_start_emb, W4_hj_end_emb)
        h_ij = torch.tanh(span)
        return h_ij


class InterpretationModel(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.h_t = nn.Linear(hidden_size, 1)

    def forward(self, h_ij, span_masks):
        o_ij = self.h_t(h_ij).squeeze(-1)  # (ba, span_num)
        # mask illegal span
        o_ij = o_ij - span_masks
        # normalize all a_ij, a_ij sum = 1
        a_ij = nn.functional.softmax(o_ij, dim=1)
        # weight average span representation to get H
        H = (a_ij.unsqueeze(-1) * h_ij).sum(dim=1)  # (bs, hidden_size)
        return H, a_ij


# def main():
#     # data
#     input_id_1 = torch.LongTensor([0, 4, 5, 6, 7, 2])
#     input_id_2 = torch.LongTensor([0, 4, 5, 2])
#     input_id_3 = torch.LongTensor([0, 4, 2])
#     batch = [(input_id_1, torch.LongTensor([1]), torch.LongTensor([6])),
#              (input_id_2, torch.LongTensor([1]), torch.LongTensor([4])),
#              (input_id_3, torch.LongTensor([1]), torch.LongTensor([3]))]

#     output = collate_to_max_length(batch=batch, fill_values=[1, 0, 0])
#     input_ids, labels, length, start_indexs, end_indexs, span_masks = output

#     # model
#     bert_path = "roberta-base"
#     model = ExplainableModel(bert_path)
#     print(model)

#     output = model(input_ids, start_indexs, end_indexs, span_masks)
#     print(output[0])


# if __name__ == '__main__':
#     main()


In [ ]:
model = ExplainableModel(args.bert_model)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

ExplainableModel(
  (intermediate): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

# Train

In [ ]:
def main():

    # processors = {
    #     "fever": FeverProcessor,
    # }

    output_modes = {
        "fever": "classification",
    }

    # Huấn luyện song song trên GPU
    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        n_gpu = 1
        # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.distributed.init_process_group(backend='nccl')

    # Xuất thông báo khởi tạo môi trường cho việc huấn luyện
    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN)

    logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args.local_rank != -1), args.fp16))

    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
            args.gradient_accumulation_steps))

    args.train_batch_size = args.train_batch_size // args.gradient_accumulation_steps

    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

    if not args.do_train and not args.do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")

    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train:
        raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    task_name = args.task_name.lower()

    # if task_name not in processors:
    #     raise ValueError("Task not found: %s" % (task_name))

    # processor = processors[task_name]()
    output_mode = output_modes[task_name]

    label_list = ["NON", "SR"]
    num_labels = len(label_list)

    tokenizer = AutoTokenizer.from_pretrained(args.bert_model, use_fast=False)
    # tokenizer = AutoTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)

    train_examples = None
    num_train_optimization_steps = None

    # Prepare model
    cache_dir = args.cache_dir if args.cache_dir else os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE),
                                                                   'distributed_{}'.format(args.local_rank))
    model = ExplainableModel(args.bert_model)

    # DEBUG
    # model.load_state_dict(torch.load(args.evidence_path))
    # model.cuda()

    if args.fp16:
        model.half()
    model.to(device)
    if args.local_rank != -1:
        try:
            from apex.parallel import DistributedDataParallel as DDP
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

        model = DDP(model)
    elif n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Prepare optimizer
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters,
                        lr=args.learning_rate)

    global_step = 0
    nb_tr_steps = 0
    tr_loss = 0

    logger.info(f"Optimzer:{optimizer.__class__.__name__}")

    if args.do_train:
        kfold = KFold(n_splits=args.kfold, shuffle=True, random_state=42)

        train_dataset_pos = DSC_Dataset(directory=args.data_dir, prefix="train_pos", bert_path=args.bert_model, max_length=args.max_seq_length)

        train_dataset_neg = DSC_Dataset(directory=args.data_dir, prefix="train_neg", bert_path=args.bert_model, max_length=args.max_seq_length)

        num_train_optimization_steps = int(
            len(train_dataset_pos) / args.train_batch_size / args.gradient_accumulation_steps) * args.num_train_epochs

        # logger.info(f"\nNum Train Optimization Steps: {num_train_optimization_steps}\n")

        if args.losstype == 'cross_entropy_concat' or args.losstype == 'hinge_loss_concat':
            num_train_optimization_steps = int((len(train_dataset_pos)+len(train_dataset_neg)) / args.train_batch_size / args.gradient_accumulation_steps) * args.num_train_epochs

        if args.local_rank != -1:
            num_train_optimization_steps = num_train_optimization_steps // torch.distributed.get_world_size()


        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(train_dataset_pos))
        logger.info("  Batch size = %d", args.train_batch_size)
        logger.info("  Num steps = %d", num_train_optimization_steps)

        pos_fold = {"train": [], "test": []}
        for fold_train, fold_test in kfold.split(train_dataset_pos):
            pos_fold["train"].append(fold_train)
            pos_fold["test"].append(fold_test)

        neg_fold = {"train": [], "test": []}
        for fold_train, fold_test in kfold.split(train_dataset_neg):
            neg_fold["train"].append(fold_train)
            neg_fold["test"].append(fold_test)


        for epoch in trange(int(args.num_train_epochs), desc="Epoch"):
            model.train()
            label_pred = []
            label_true = []
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            idx_train_pos = pos_fold["train"][epoch]
            idx_train_neg = neg_fold["train"][epoch]
            idx_eval_pos = pos_fold["test"][epoch]
            idx_eval_neg = neg_fold["test"][epoch]

            train_sampler_pos = SubsetRandomSampler(idx_train_pos)
            train_sampler_neg = SubsetRandomSampler(idx_train_neg)

            # train_dataloader_pos = DataLoader(train_data_pos, sampler=train_sampler_pos, batch_size=args.train_batch_size)
            # train_dataloader_neg = DataLoader(train_data_neg, sampler=train_sampler_neg, batch_size=args.negative_batch_size)

            train_dataloader_pos = get_feature(dataset=train_dataset_pos,sampler=train_sampler_pos, batch_size = args.train_batch_size)
            train_dataloader_neg = get_feature(dataset=train_dataset_neg,sampler=train_sampler_neg, batch_size = args.train_batch_size)


            # eval_sampler = SubsetRandomSampler(pos_fold["test"][epoch])
            # eval_neg_sampler = SubsetRandomSampler(neg_fold["test"][epoch])
            eval_data = ConcatDataset([Subset(train_dataset_pos,idx_eval_pos),Subset(train_dataset_neg, idx_eval_neg)])
            eval_sampler = RandomSampler(eval_data)
            eval_dataloader = get_feature(dataset=eval_data,sampler=eval_sampler, batch_size = args.eval_batch_size)
            it=iter(train_dataloader_neg)
            for step, batch in enumerate(tqdm(train_dataloader_pos, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, label, length, start_index, end_index, span_mask = batch

                # define a new function to compute loss values for both output_modes
                try:
                    batch_neg = tuple(t.to(device) for t in next(it))
                except:
                    it = iter(train_dataloader_neg)
                    batch_neg = tuple(t.to(device) for t in next(it))
                input_ids_neg, label_neg, length_neg, start_index_neg, end_index_neg, span_mask_neg = batch_neg

                input_ids_cat=torch.cat([input_ids, input_ids_neg],dim=0)
                start_index_cat=torch.cat([start_index, start_index_neg],dim=0)
                end_index_cat=torch.cat([end_index,end_index_neg],dim=0)
                span_mask_cat=torch.cat([span_mask,span_mask_neg],dim=0)
                label_cat=torch.cat([label.view(-1), label_neg.view(-1)], dim = 0)

                with torch.no_grad():
                    output_logits = model(input_ids=input_ids_cat, start_indexs=start_index_cat, end_indexs=end_index_cat, span_masks=span_mask_cat)

                if output_mode == "classification":
                    loss_fct = CrossEntropyLoss()
                    if args.losstype == 'cross_entropy':
                        loss = loss_fct(output_logits.logits.view(-1, num_labels),label_cat)

                    elif args.losstype == 'cross_entropy_mining':

                        loss_fct = CrossEntropyLoss(reduction='none')
                        loss = loss_fct(output_logits.logits.view(-1, num_labels),label_cat)
                        NON_index = label_cat.view(-1) == 1
                        SR_index = label_cat.view(-1) != 1
                        Pos_numbers = SR_index.sum()
                        Top_numbers = Pos_numbers * args.scale_pos_neg

                        TOP_K, Hard_NON_index = loss[NON_index].topk(Top_numbers)
                        # loss = torch.cat([loss[SR_index], TOP_K])    #uncomment if dont want to use no-grad then grad

                        #comment if dont want to use no-grad then grad
                        IDS = torch.cat([torch.tensor(range(0,Pos_numbers),device=device), Pos_numbers+Hard_NON_index], dim=0).to(torch.int32)
                        output_logits = model(input_ids=input_ids_cat[IDS, :],start_indexs=start_index_cat[IDS, :], end_indexs=end_index_cat[IDS, :], span_masks=span_mask_neg[IDS, :])

                        # Compute loss, evaluation
                        probs = torch.nn.functional.softmax(output_logits.logits, dim=-1)
                        _, pred = torch.max(probs, dim=-1)

                        label_pred.extend(pred.cpu().numpy())
                        label_true.extend(label_cat[IDS].cpu().numpy())

                        # loss_fct = CrossEntropyLoss()
                        loss_fct = CrossEntropyLoss(weight=torch.tensor([args.scale_pos_neg,1.]).to(device))
                        loss = loss_fct(output_logits.logits.view(-1, num_labels), label_cat[IDS])

                elif output_mode == "regression":
                    loss_fct = MSELoss()
                    loss = loss_fct(output_logits.logits.view(-1), label_cat)

                if n_gpu > 1:
                    loss = loss.mean()  # mean() to average on multi-gpu.
                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps

                if args.fp16:
                    optimizer.backward(loss)
                else:
                    loss.backward()

                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1

                if (step + 1) % args.gradient_accumulation_steps == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                    global_step += 1
            print('printing loss')
            print('training_loss~=',tr_loss/nb_tr_steps)
            print(acc_and_f1(preds=label_pred, labels=label_true))


            if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
                # Save a trained model, configuration and tokenizer
                model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

                # If we save using the predefined names, we can load using `from_pretrained`
                output_model_file = os.path.join(args.output_dir, f"{WEIGHTS_NAME}_{epoch}")
                output_config_file = os.path.join(args.output_dir, CONFIG_NAME)

                torch.save(model_to_save.state_dict(), output_model_file)
                if epoch == 0:
                    tokenizer.save_vocabulary(args.output_dir)

            model.eval()
            eval_loss = 0
            nb_eval_steps = 0
            preds = []
            preds = []
            labels = []
            probs = []
            store_output=list()
            # softmaxing=torch.nn.Softmax()
            for input_ids, label, length, start_index, end_index, span_mask in tqdm(eval_dataloader, desc="Evaluating"):
                input_ids = input_ids.to(device)
                start_index = start_index.to(device)
                end_index = end_index.to(device)
                span_mask = span_mask.to(device)
                label_ids = label.to(device)
                #
                model.zero_grad()
                with torch.no_grad():
                    # output_logits = model(input_ids, segment_ids, input_mask, labels=None)
                    output_logits = model(input_ids=input_ids_cat, start_indexs=start_index_cat, end_indexs=end_index_cat, span_masks=span_mask_cat)

                    # logits=softmaxing(logits)
                    store_output.extend(output_logits.logits.cpu().numpy())
                    prob_in_batch = torch.nn.functional.softmax(output_logits.logits, dim=-1)

                    # Chọn nhãn với xác suất cao nhất
                    prob, pred = torch.max(prob_in_batch, dim=-1)
                    preds.extend(pred.cpu().numpy())
                    labels.extend(label_ids.cpu().numpy())
                    probs.extend(prob.cpu().numpy())
            print(acc_and_f1(preds=preds, labels=labels))

            # if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
            #     # Save a trained model, configuration and tokenizer
            #     model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

            #     # If we save using the predefined names, we can load using `from_pretrained`
            #     output_model_file = os.path.join(args.output_dir, f"{WEIGHTS_NAME}_{epoch}")
            #     output_config_file = os.path.join(args.output_dir, CONFIG_NAME)

            #     torch.save(model_to_save.state_dict(), output_model_file)
            #     if epoch == 0:
            #         tokenizer.save_vocabulary(args.output_dir)


    # if args.do_test and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
    #   # model.load_state_dict(torch.load(os.path.join(args.output_dir,f"pytorch_model_{epoch}.bin")))
    #     test_examples = processor.get_test_examples(args.data_dir)
    #     # test_examples=test_examples[0:500] #debugging
    #     test_features = convert_examples_to_features_test(
    #         test_examples, label_list, args.max_seq_length, tokenizer, output_mode)
    #     logger.info("***** Running testing *****")
    #     logger.info("  Num examples = %d", len(test_examples))
    #     logger.info("  Batch size = %d", args.eval_batch_size)
    #     all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    #     all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    #     all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

    #     test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    #     # Run prediction for full data
    #     test_sampler = SequentialSampler(test_data)
    #     test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.eval_batch_size)

    #     model.eval()
    #     preds = []
    #     probs = []
    #     store_output=list()
    #     # softmaxing=torch.nn.Softmax()
    #     for input_ids, input_mask, segment_ids in tqdm(test_dataloader, desc="Evaluating"):
    #         input_ids = input_ids.to(device)
    #         input_mask = input_mask.to(device)
    #         segment_ids = segment_ids.to(device)

    #         with torch.no_grad():
    #             output_logits = model(input_ids=input_ids, token_type_ids=None, attention_mask=input_mask, labels=None)
    #             store_output.extend(output_logits.logits.cpu().numpy())
    #             prob_in_batch = torch.nn.functional.softmax(output_logits.logits, dim=-1)

    #             # Chọn nhãn với xác suất cao nhất
    #             prob, pred = torch.max(prob_in_batch, dim=-1)
    #             preds.extend(pred.cpu().numpy())
    #             probs.extend(prob.cpu().numpy())
    #     # torch.from_numpy(np.array(store_output))
    #     print("Storing dev scores")
    #     claim_ids = []
    #     num_sentences = []
    #     claims =[]
    #     evidences = []
    #     ids = []
    #     for example in test_examples:
    #         id, claim_id, num_sentence, claim, evidence = processor.get_info_eval(example)
    #         ids.append(id)
    #         claim_ids.append(claim_id)
    #         num_sentences.append(num_sentence)
    #         claims.append(claim)
    #         evidences.append(evidence)

    #     data = {
    #         "claim_id": claim_ids,
    #         "num_sentence": num_sentences,
    #         "claim": claims,
    #         "evidence": evidences,
    #         "pred": preds,
    #         "prob": probs
    #     }
    #     eval_df = pd.DataFrame(data=data)
    #     eval_df.to_json(os.path.join(args.output_dir,"test_non_evidence.json"), force_ascii=False, indent=4, orient='index')


if __name__ == "__main__":
    main()

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6548 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:05<?, ?it/s]


RuntimeError: ignored